In [1]:
import pandas as pd
import numpy as np

### LOAD Model Files
modelpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/recipe-difficulty-tagger_App/models/'
modelpicklepath = modelpath+"bbcgfd_ilda_tf_100Ingredtopics.pkl"

### LOAD CSV File
staticpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/recipe-difficulty-tagger_App/static/'
recipeListpath = staticpath + "bbcgdfood_recipelist4model_recipe-diff-tag_app.csv"

recipeDF = pd.read_csv(recipeListpath,index_col=0)


In [3]:
# recipeDF.ix[9199]

In [4]:
DiffN = recipeDF.reset_index().prepDifficulty.to_frame()
DiffN
DiffN[DiffN=='Easy'] = 1
DiffN[DiffN=='More effort'] = 2
DiffN[DiffN=='A challenge'] = 3
# test.prepDifficulty.unique()
recipeDF['prepDiffN'] = DiffN
# recipeDF

In [5]:
## Retrieve NLP / LDA Models

# load nltk's SnowballStemmer as variabled 'stemmer'
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# define here a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    #tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens


## -- this part above still needs to be present outside of the one below to load it via pickle ----------------
## -------------------------------------------------------------------------------------------------

### RETRIEVE OUTPUTS
import pickle

# # ilda_tf, Ingdtm_tf, itf_vectorizer, pyldavisI
# with open('bbcgfd_ilda_tf_100Ingredtopics.pkl', 'wb') as f:
#     pickle.dump(ilda_tf, f) #model
#     pickle.dump(Ingdtm_tf, f) #doc-token-vec
#     pickle.dump(itf_vectorizer, f) #vectorizer
#     pickle.dump(pyldavisItf, f) #pyLDAvis

# # mlda_tf, Methoddtm_tf, mtf_vectorizer, pyldavisM
# with open('bbcgfd_mlda_tf_80methodtopics.pkl', 'wb') as f:
#     pickle.dump(mlda_tf, f)
#     pickle.dump(Methoddtm_tf, f)
#     pickle.dump(mtf_vectorizer, f)
#     pickle.dump(pyldavisMtf, f)

    
with open(modelpicklepath, 'rb') as f:
    
    ilda_tf = pickle.load(f) #model
    Ingdtm_tf= pickle.load(f) #doc-token-vec
    itf_vectorizer= pickle.load(f) #vectorizer
    pyldavisItf= pickle.load(f) #pyLDAvis


In [6]:
import re

## USE ingredient LDA to find similar recipes:
iX_vec = itf_vectorizer.transform(recipeDF.ingred)
iX_ldaAtrans = pd.DataFrame(ilda_tf.transform(iX_vec))

from sklearn.neighbors import KDTree

# simtree = KDTree(np.array(iX_ldaAtrans), metric='euclidean' )
simtree = KDTree(np.array(iX_ldaAtrans), metric='euclidean' )
# simtree.valid_metrics
# ['l2',
#  'minkowski',
#  'l1',
#  'p',
#  'infinity',
#  'euclidean',
#  'chebyshev',
#  'manhattan',
#  'cityblock']


In [327]:
recipeID = 47

In [328]:
dist,ind = simtree.query(np.array(iX_ldaAtrans.iloc[recipeID]),k=100)

/Users/hrm/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [329]:
## Use Set Theory to exclude weirdly related ones

simList0 = recipeDF.ix[ind.tolist()[0]]
simList0['KNNdist'] = dist.tolist()[0]
simList = simList0#[1:] 
simList
# simList.sort_values(by=['title','KNNdist'])
# simList[simList.prepDiffN==1].sort_values(by="KNNdist").head(3)

,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN,KNNdist
47,47,Tomato soup,"Firstly, prepare your vegetables. You need 1-1...",ripe tomatoes onion carrot celery olive oil to...,9,15.0,25.0,Easy,http://www.bbcgoodfood.com/recipes/2075/tomato...,http://www.bbcgoodfood.com/sites/default/files...,1,0.000000
6248,6265,Caramelised shallots,Heat oil in a large frying pan and cook shallo...,vegetable oil shallots onion butter bay leaves...,6,10.0,30.0,Easy,http://www.bbcgoodfood.com/recipes/1132633/car...,http://www.bbcgoodfood.com/sites/default/files...,1,2.451072
1556,1557,Sausage & bean casserole,"Heat half the oil in a large casserole dish, t...",vegetable oil sausage celery sticks carrot oni...,11,15.0,50.0,Easy,http://www.bbcgoodfood.com/recipes/1649635/sau...,http://www.bbcgoodfood.com/sites/default/files...,1,3.056909
7009,7027,Pan-fried venison with sloe gin & plum sauce,Press the juniper and lots of black pepper ont...,juniper berry Finely chopped venison steaks bu...,7,20.0,20.0,Easy,http://www.bbcgoodfood.com/recipes/pan-fried-v...,http://www.bbcgoodfood.com/sites/default/files...,1,3.207779
285,286,Somerset stew with cheddar & parsley mash,"Heat oil in a large pan and fry the onion, gar...",oil onion garlic clove carrot leek tomato puré...,14,15.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/1089635/som...,http://www.bbcgoodfood.com/sites/default/files...,1,3.292585
5800,5817,Tomato & rice soup,Heat the oil in a large saucepan and add the o...,olive oil onion carrot celery stick sugar vine...,9,10.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/tomato-rice...,http://www.bbcgoodfood.com/sites/default/files...,1,3.301414
3279,3287,Low-fat turkey Bolognese,Heat a large non-stick frying pan and dry-fry ...,turkey mince vegetable oil onion carrot celery...,10,10.0,45.0,Easy,http://www.bbcgoodfood.com/recipes/low-fat-tur...,http://www.bbcgoodfood.com/sites/default/files...,1,3.545558
2682,2685,Velvety duck liver parfait,Cut away and discard any large sinews from the...,duck liver butter shallot onion garlic brandy ...,10,30.0,15.0,Easy,http://www.bbcgoodfood.com/recipes/4971/velvet...,http://www.bbcgoodfood.com/sites/default/files...,1,3.697672
5416,5432,"More veg, less meat summer Bolognese",Heat the oil in large deep frying pan. Add the...,olive oil onion carrot celery sticks courgette...,12,15.0,40.0,Easy,http://www.bbcgoodfood.com/recipes/2320653/mor...,http://www.bbcgoodfood.com/sites/default/files...,1,3.877593
5359,5375,Beef fillet with horseradish & prosciutto,Heat the butter in a large frying pan and fry ...,butter beef fillet shallots horseradish sauce ...,11,20.0,60.0,More effort,http://www.bbcgoodfood.com/recipes/beef-fillet...,http://www.bbcgoodfood.com/sites/default/files...,2,3.894066


In [254]:
indx0 = set(simList.index.tolist()) 
indx = indx0.difference([47])
# indx

In [179]:
ref = set(simList.ix[recipeID].title.lower().replace('&','').replace('and','').replace('with','').split())
ref

{'chestnut', 'mushroom', 'roast', 'stuffed', 'three-bird', 'wild'}

In [180]:
test = set(simList.ix[10038].title.lower().replace('&','').replace('and','').replace('with','').split())
test

{'gravy', 'make-ahead', 'mushroom', 'prosecco', 'wild'}

In [121]:
ref.intersection(test)

set()

In [446]:
indx0 = set(simList.index.tolist()) 
indx = indx0.difference([recipeID])
ref = set(simList.ix[recipeID].title.lower().replace('&','').replace('and','').replace('with','').split())

keepRidx=[]
for ridx in indx:
    test = set(simList.ix[ridx].title.lower().replace('&','').replace('and','').replace('with','').split())
    
    if len(ref.intersection(test))>=1:
        keepRidx.append(ridx)

# keepRidx    

In [285]:
ref = set(simList.ix[recipeID].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())
ref

{'chicken',
 'chillies',
 'ginger',
 'peeled',
 'prawns',
 'raw',
 'red',
 'sauce',
 'small',
 'soy',
 'spring-onions',
 'stock',
 'sugar',
 'vinegar'}

In [287]:
test = set(simList.ix[4443].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())
test

{'beansprout',
 'egg',
 'oil',
 'sauce,',
 'serve',
 'soy',
 'spring-onion',
 'to',
 'vegetable'}

In [288]:
ref.intersection(test)

{'soy'}

In [289]:
len(ref.intersection(test))/len(ref)

0.07142857142857142

In [314]:
len(ref.intersection(test))/len(ref)

0.21428571428571427

In [336]:
indx0 = set(simList.index.tolist()) 
indx = indx0.difference([recipeID])
ref = set(simList.ix[recipeID].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())

keepRidx = []
Intersectpcent = []
for ridx in indx:
    test = set(simList.ix[ridx].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())
    
    Intersectpcent.append(len(ref.intersection(test))/len(ref))

for idx in range(len(Intersectpcent)):    
    if Intersectpcent[idx]>=np.max(Intersectpcent)*0.8:
        keepRidx.append(list(indx)[idx])

keepRidx 
# np.max(Intersectpcent)*0.8

[104, 5800, 1721]

In [310]:
# test

In [293]:
simList.ix[keepRidx].sort_values(by="KNNdist")

,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN,KNNdist
4217,4229,Fried rice with egg & ginger,Heat the oil in a wok. Add the ginger and garl...,sunflower oil ginger garlic small peeled cooke...,8,15.0,15.0,Easy,http://www.bbcgoodfood.com/recipes/1805661/fri...,http://www.bbcgoodfood.com/sites/default/files...,1,2.550049
1152,1153,Egg fried rice with prawns & peas,Put the rice in a saucepan with 600ml water. B...,basmati rice vegetable oil garlic red chilli e...,9,5.0,20.0,Easy,http://www.bbcgoodfood.com/recipes/11189/egg-f...,http://www.bbcgoodfood.com/sites/default/files...,1,2.805663
3443,3451,Thai prawn noodles,"Cook the noodles following pack instructions, ...",egg noodles vegetable oil ginger garlic red ch...,12,20.0,10.0,Easy,http://www.bbcgoodfood.com/recipes/thai-prawn-...,http://www.bbcgoodfood.com/sites/default/files...,1,3.893126
1303,1304,Grilled Thai salmon,Heat grill to high. Place the fish in a shallo...,salmon fillets sunflower oil ginger red chilli...,8,5.0,10.0,Easy,http://www.bbcgoodfood.com/recipes/4774/grille...,http://www.bbcgoodfood.com/sites/default/files...,1,3.996874
9169,9197,Aubergine with prawns in oyster sauce,"Heat the 3 tbsp oil in a wok or large, non-sti...",sunflower oil aubergine red pepper spring-onio...,9,10.0,20.0,Easy,http://www.bbcgoodfood.com/recipes/aubergine-p...,http://www.bbcgoodfood.com/sites/default/files...,1,4.160031
8144,8166,Cold chicken noodle salad,"Shred the meat from the leftover chicken legs,...",leftover cooked chicken legs soba noodles wasa...,10,15.0,15.0,Easy,http://www.bbcgoodfood.com/recipes/cold-chicke...,http://www.bbcgoodfood.com/sites/default/files...,1,4.164632
7775,7797,Congee with soy eggs,First make the soy eggs. Bring a small pan of ...,sushi rice chicken stock butter ginger spring-...,11,10.0,25.0,Easy,http://www.bbcgoodfood.com/recipes/congee-soy-...,http://www.bbcgoodfood.com/sites/default/files...,1,4.240155
8884,8912,Beef noodle salad with stem ginger dressing,"In a mixing bowl, combine the ingredients for ...",cooked egg noodle sugar snap pea radish thumb-...,10,20.0,25.0,Easy,http://www.bbcgoodfood.com/recipe/beef-noodle-...,http://www.bbcgoodfood.com/sites/default/files...,1,4.412056
6593,6611,Steamed lemon tilapia with teriyaki sauce,"Pour the soy, mirin and sugar in a small sauce...",soy sauce sugar lemon basmati rice /5oz pieces...,8,10.0,20.0,Easy,http://www.bbcgoodfood.com/recipes/1364633/ste...,http://www.bbcgoodfood.com/sites/default/files...,1,4.550452
9046,9074,Citrus & ginger steamed fish with stir-fry veg,"Mix together the orange zest and juice, soy an...",orange soy sauce vinegar fish ginger sesame oi...,11,10.0,10.0,Easy,http://www.bbcgoodfood.com/recipes/citrus-ging...,http://www.bbcgoodfood.com/sites/default/files...,1,4.662998


In [27]:
# tmp = recipeDF.ix[ind.tolist()[0]]
# tmp['KNNdist'] = dist.tolist()[0]
# tmp[1:]#.sort_values(by="KNNdist")
# # tmp[1:].sort_values(by=['KNNdist','prepDifficulty'])
# easyOUTlist = simList[simList.prepDiffN==1].sort_values(by="KNNdist")#.head(3)
# easyOUTlist[easyOUTlist.KNNdist<3.5].head()
# diffOUTlist = simList[simList.prepDiffN!=1].sort_values(by="KNNdist")#.head(ChooseN)
# diffOUTlist[diffOUTlist.KNNdist<5]


,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN,KNNdist


In [182]:
## Original
# def getN_easyNdiff_similarRecipes(recipeLookupIDX,Knum,ChooseN):
#     dist,ind = simtree.query(np.array(iX_ldaAtrans.iloc[recipeLookupIDX]),k=Knum)
    
#     simList0 = recipeDF.ix[ind.tolist()[0]] #[1:]]
#     simList0['KNNdist'] = dist.tolist()[0]
#     simList = simList0[1:]                          
#     #simList
#     ## Might want to add a few additioanl conditions for selection -- based on a cosine distance threshold 
#     ## OR use word2vec to finetune the 'in-out' categories...    
    
#     easyOUTlist = simList[simList.prepDiffN==1].sort_values(by=["KNNdist","title"]).head(ChooseN)
#     diffOUTlist = simList[simList.prepDiffN!=1].sort_values(by=["KNNdist","title"]).head(ChooseN)
    
#     return easyOUTlist, diffOUTlist
    

In [193]:
# ## Adds Title tokens as filter

# def getN_easyNdiff_similarRecipes(recipeLookupIDX,Knum,ChooseN):
    
#     dist,ind = simtree.query(np.array(iX_ldaAtrans.iloc[recipeLookupIDX]),k=Knum)
    
#     simList0 = recipeDF.ix[ind.tolist()[0]] #[1:]]
#     simList0['KNNdist'] = dist.tolist()[0]
#     simList = simList0#[1:]                          
#     #simList
#     ## Might want to add a few additioanl conditions for selection -- based on a cosine distance threshold 
#     ## OR use word2vec to finetune the 'in-out' categories...    
    
    
#     indx0 = set(simList.index.tolist()) 
#     indx = indx0.difference([recipeLookupIDX])
#     ref = set(simList.ix[recipeLookupIDX].title.lower()
#               .replace('&','')
#               .replace('and','')
#               .replace('with','')
#               .replace('The ultimate makeover:','')
#               .split())

#     keepRidx=[]
#     for ridx in indx:
#         test = set(simList.ix[ridx].title.lower()
#               .replace('&','')
#               .replace('and','')
#               .replace('with','')
#               .replace('The ultimate makeover:','')
#               .split())

#         #if len(ref.intersection(test))>=2:
#         if len(ref.intersection(test)) / len(ref) >= 0.3:
#             if "gravy" in ref and 'gravy' in test:
#                 keepRidx.append(ridx)
#             elif "gravy" not in ref and 'gravy' not in test:
#                 keepRidx.append(ridx)
#             elif "gravy" not in ref and 'gravy' in test:
#                 continue
             
                
# #         elif len(ref.intersection(test))>=1:
# #             keepRidx.append(ridx)

#     simList2 = simList.ix[keepRidx].sort_values(by="KNNdist")
    
#     easyOUTlist = simList2[simList2.prepDiffN==1].sort_values(by=["KNNdist","title"]).head(ChooseN)
#     diffOUTlist = simList2[simList2.prepDiffN!=1].sort_values(by=["KNNdist","title"]).head(ChooseN)
    
#     return easyOUTlist, diffOUTlist, simList2, simList
    

In [441]:
## Adds Ingredient tokens as filter

def getN_easyNdiff_similarRecipes(recipeLookupIDX,Knum,ChooseN):
    
    dist,ind = simtree.query(np.array(iX_ldaAtrans.iloc[recipeLookupIDX]),k=Knum)
    
    simList0 = recipeDF.ix[ind.tolist()[0]] #[1:]]
    simList0['KNNdist'] = dist.tolist()[0]
    simList = simList0#[1:]                          
    #simList
    ## Might want to add a few additioanl conditions for selection -- based on a cosine distance threshold 
    ## OR use word2vec to finetune the 'in-out' categories...    
    
    
    indx0 = set(simList.index.tolist()) 
    indx = indx0.difference([recipeLookupIDX])
    ref = set(simList.ix[recipeLookupIDX].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())

    keepRidx=[]
#     for ridx in indx:
#         test = set(simList.ix[ridx].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())

#         if len(ref.intersection(test))/len(ref)>=0.3:
#             keepRidx.append(ridx)
    
    Intersectpcent = []
    for ridx in indx:
        test = set(simList.ix[ridx].ingred.lower().replace('&','').replace('and','').replace('with','').replace('of','').split())
    
        Intersectpcent.append(len(ref.intersection(test))/len(ref))

    for idx in range(len(Intersectpcent)):    
        if Intersectpcent[idx]>=np.max(Intersectpcent)*0.5:
            keepRidx.append(list(indx)[idx])


    simList2 = simList.ix[keepRidx].sort_values(by="KNNdist")
    
    easyOUTlist = simList2[simList2.prepDiffN==1].sort_values(by=["KNNdist","title"]).head(ChooseN)
    diffOUTlist = simList2[simList2.prepDiffN!=1].sort_values(by=["KNNdist","title"]).head(ChooseN)
#     easyOUTlist = simList2[simList2.prepDiffN==1].sort_values(by=["KNNdist"]).head(ChooseN)
#     diffOUTlist = simList2[simList2.prepDiffN!=1].sort_values(by=["KNNdist"]).head(ChooseN)
    
    return easyOUTlist, diffOUTlist, simList2, simList
    

In [442]:
# easy, diff = getN_easyNdiff_similarRecipes(recipeLookupIDX,Knum)
easy, diff, simList2, simList = getN_easyNdiff_similarRecipes(47,100,3)

/Users/hrm/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [421]:
# keepRidx

In [443]:
simList2

,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN,KNNdist
1556,1557,Sausage & bean casserole,"Heat half the oil in a large casserole dish, t...",vegetable oil sausage celery sticks carrot oni...,11,15.0,50.0,Easy,http://www.bbcgoodfood.com/recipes/1649635/sau...,http://www.bbcgoodfood.com/sites/default/files...,1,3.056909
285,286,Somerset stew with cheddar & parsley mash,"Heat oil in a large pan and fry the onion, gar...",oil onion garlic clove carrot leek tomato puré...,14,15.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/1089635/som...,http://www.bbcgoodfood.com/sites/default/files...,1,3.292585
5800,5817,Tomato & rice soup,Heat the oil in a large saucepan and add the o...,olive oil onion carrot celery stick sugar vine...,9,10.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/tomato-rice...,http://www.bbcgoodfood.com/sites/default/files...,1,3.301414
3279,3287,Low-fat turkey Bolognese,Heat a large non-stick frying pan and dry-fry ...,turkey mince vegetable oil onion carrot celery...,10,10.0,45.0,Easy,http://www.bbcgoodfood.com/recipes/low-fat-tur...,http://www.bbcgoodfood.com/sites/default/files...,1,3.545558
5416,5432,"More veg, less meat summer Bolognese",Heat the oil in large deep frying pan. Add the...,olive oil onion carrot celery sticks courgette...,12,15.0,40.0,Easy,http://www.bbcgoodfood.com/recipes/2320653/mor...,http://www.bbcgoodfood.com/sites/default/files...,1,3.877593
6484,6501,Make-ahead gravy,Heat oven to 220C/200C fan/gas 7. Tip the chic...,chicken wing turkey neck carrot onion celery s...,12,10.0,70.0,Easy,http://www.bbcgoodfood.com/recipes/2743668/mak...,http://www.bbcgoodfood.com/sites/default/files...,1,4.219898
2429,2432,Pork & rosemary lasagne,Preheat the oven to 190C/Gas 5/fan oven 170C. ...,olive oil pork onion celery rosemary white win...,15,40.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/1235/pork-a...,http://www.bbcgoodfood.com/sites/default/files...,1,4.290008
1480,1481,Minced beef & sweet potato stew,"Heat the oil in a large pan, add the onion, ca...",sunflower oil onion carrot celery stick beef t...,13,20.0,60.0,Easy,http://www.bbcgoodfood.com/recipes/2778668/min...,http://www.bbcgoodfood.com/sites/default/files...,1,4.446765
164,164,Beef & stout stew with carrots,Heat oven to 160C/140C fan/gas 3. Heat the oil...,vegetable oil beef onion carrot plain flour Gu...,11,30.0,180.0,Easy,http://www.bbcgoodfood.com/recipes/13349/beef-...,http://www.bbcgoodfood.com/sites/default/files...,1,4.473459
104,104,Beef & vegetable casserole,Heat oven to 160C/140C fan/gas 3. Put the kett...,celery sticks onion carrot bay leaf sprigs veg...,12,15.0,230.0,Easy,http://www.bbcgoodfood.com/recipes/1940689/bee...,http://www.bbcgoodfood.com/sites/default/files...,1,4.526583


In [444]:
easy

,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN,KNNdist
1556,1557,Sausage & bean casserole,"Heat half the oil in a large casserole dish, t...",vegetable oil sausage celery sticks carrot oni...,11,15.0,50.0,Easy,http://www.bbcgoodfood.com/recipes/1649635/sau...,http://www.bbcgoodfood.com/sites/default/files...,1,3.056909
285,286,Somerset stew with cheddar & parsley mash,"Heat oil in a large pan and fry the onion, gar...",oil onion garlic clove carrot leek tomato puré...,14,15.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/1089635/som...,http://www.bbcgoodfood.com/sites/default/files...,1,3.292585
5800,5817,Tomato & rice soup,Heat the oil in a large saucepan and add the o...,olive oil onion carrot celery stick sugar vine...,9,10.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/tomato-rice...,http://www.bbcgoodfood.com/sites/default/files...,1,3.301414


In [445]:
diff

,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN,KNNdist
1706,1707,Carbonnade flamande,Marinate the beef overnight in the ale with th...,beef Trappist ale such as garlic clove bay lea...,13,90.0,120.0,More effort,http://www.bbcgoodfood.com/recipes/1236/carbon...,http://www.bbcgoodfood.com/sites/default/files...,2,4.731293


In [11]:
import os

# !cd recipe-difficulty-tagger_App/
# os.chdir(os.path.dirname('./ingredient-phrase-tagger-master/'))
!pwd

/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/ingredient-phrase-tagger-master


In [12]:
# import getSimilar
# from getSimilar import getN_easyNdiff_similarRecipes

# recipeDF = getSimilar.recipeDF

recipeDF

,index,title,method,ingred,Ningred,prepTime,cookTime,prepDifficulty,recipeLink,imgURL,prepDiffN
0,0,Lemon drizzle cake,Heat oven to 180C/fan 160C/gas 4. Beat togethe...,unsalted butter sugar egg lemon self-raising f...,7,15.0,45.0,Easy,http://www.bbcgoodfood.com/recipes/4942/lemon-...,http://www.bbcgoodfood.com/sites/default/files...,1
1,1,Chilli con carne,Prepare your vegetables. Chop 1 large onion in...,oil onion red pepper garlic hot chilli powder ...,16,70.0,25.0,Easy,http://www.bbcgoodfood.com/recipes/3228/chilli...,http://www.bbcgoodfood.com/sites/default/files...,1
2,2,Ultimate chocolate cake,Butter a 20cm round cake tin (7.5cm deep) and ...,dark-chocolate butter instant coffee granules ...,15,40.0,90.0,Easy,http://www.bbcgoodfood.com/recipes/3092/ultima...,http://www.bbcgoodfood.com/sites/default/files...,1
3,3,Best-ever brownies,Cut 185g unsalted butter into smallish cubes a...,unsalted butter best dark-chocolate plain flou...,8,15.0,25.0,More effort,http://www.bbcgoodfood.com/recipes/1223/bestev...,http://www.bbcgoodfood.com/sites/default/files...,2
4,4,Yummy scrummy carrot cake,Preheat the oven to 180C/Gas 4/fan 160C. Oil a...,light muscovado sugar sunflower oil eggs raisi...,11,75.0,25.0,Easy,http://www.bbcgoodfood.com/recipes/3229/yummy-...,http://www.bbcgoodfood.com/sites/default/files...,1
5,5,Chicken & chorizo jambalaya,Heat the oil in a large frying pan with a lid ...,olive oil chicken breast onion red pepper garl...,10,10.0,45.0,Easy,http://www.bbcgoodfood.com/recipes/1167651/chi...,http://www.bbcgoodfood.com/sites/default/files...,1
6,6,Spiced carrot & lentil soup,Heat a large saucepan and dry-fry the cumin se...,cumin seeds chilli flakes olive oil red lentil...,7,10.0,15.0,Easy,http://www.bbcgoodfood.com/recipes/2089/spiced...,http://www.bbcgoodfood.com/sites/default/files...,1
7,7,Spicy root & lentil casserole,Heat the oil in a large pan and cook the onion...,oil onion garlic clove potatoes carrot parsnip...,11,10.0,35.0,Easy,http://www.bbcgoodfood.com/recipes/1364/spicy-...,http://www.bbcgoodfood.com/sites/default/files...,1
8,8,Summer-in-winter chicken,"Heat the oil in a frying pan, preferably non-s...",olive oil boneless chicken breast cherry tomat...,6,15.0,25.0,Easy,http://www.bbcgoodfood.com/recipes/1521/summer...,http://www.bbcgoodfood.com/sites/default/files...,1
9,9,Classic scones with jam & clotted cream,Heat oven to 220C/fan 200C/gas 7. Tip the flou...,self-raising flour salt baking powder butter c...,10,5.0,10.0,Easy,http://www.bbcgoodfood.com/recipes/4622/classi...,http://www.bbcgoodfood.com/sites/default/files...,1


In [27]:
# recipeDF[['index','title']].to_json('test_dropdown_recipeTitles.json')

In [14]:
import json

In [30]:
allRecipe = recipeDF[['title']].reset_index().head(10).to_json(orient='records')

In [31]:
allRecipe

'[{"index":0,"title":"Lemon drizzle cake"},{"index":1,"title":"Chilli con carne"},{"index":2,"title":"Ultimate chocolate cake"},{"index":3,"title":"Best-ever brownies"},{"index":4,"title":"Yummy scrummy carrot cake"},{"index":5,"title":"Chicken & chorizo jambalaya"},{"index":6,"title":"Spiced carrot & lentil soup"},{"index":7,"title":"Spicy root & lentil casserole"},{"index":8,"title":"Summer-in-winter chicken"},{"index":9,"title":"Classic scones with jam & clotted cream"}]'